In [3]:
# Phạm Thành Long B22ĐCN505
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

num_words = 10000
maxlen = 200

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=num_words)

# pad về cùng chiều dài
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test  = pad_sequences(x_test, maxlen=maxlen)

# ép nhãn về float32
y_train = y_train.astype("float32")
y_test  = y_test.astype("float32")

# 3. Khởi tạo layers thủ công
embedding = tf.keras.layers.Embedding(input_dim=num_words, output_dim=128)
dense1 = tf.keras.layers.Dense(32, activation="relu")
dense2 = tf.keras.layers.Dense(1)  # logits, nhị phân


# 4. Forward
def model(x):
    x = embedding(x)
    x = tf.reduce_mean(x, axis=1)  # average pooling đơn giản
    x = dense1(x)
    return dense2(x)  # logits

# 5. Loss và optimizer
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.RMSprop()

# 6. Training loop
batch_size = 128
epochs = 3
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)

for epoch in range(epochs):
    print("Epoch", epoch+1)
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, embedding.trainable_variables + dense1.trainable_variables + dense2.trainable_variables)
        optimizer.apply_gradients(zip(grads, embedding.trainable_variables + dense1.trainable_variables + dense2.trainable_variables))
    # Evaluate
    test_logits = model(x_test)
    preds = tf.round(tf.sigmoid(test_logits))
    acc = tf.reduce_mean(tf.cast(tf.equal(preds, tf.expand_dims(y_test, -1)), tf.float32))
    print("Test acc:", acc.numpy())


Epoch 1
Test acc: 0.6036
Epoch 2
Test acc: 0.65492
Epoch 3
Test acc: 0.65192
